In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import date
import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
bbg_data=pd.read_csv('Bloomberg_data.csv',header=None)
bbg_data.columns=['Source','Timestamp','Category','URL']
bbg_data.head()

,Source,Timestamp,Category,URL
0,Intelligencer,11-DEC-13-05.35.01.911037,regular,http://www.theintell.com/business/software-fir...
1,Intelligencer,12-SEP-16-11.52.34.761202,regular,http://www.theintell.com/life-style/local-feat...
2,Intelligencer,29-SEP-19-03.04.04.475263,regular,https://www.theintell.com/sports/20190827/moor...
3,Intelligencer,29-SEP-19-03.06.54.930124,regular,https://www.theintell.com/sports/20190827/hosk...
4,Intelligencer,29-SEP-19-03.18.28.775149,regular,https://www.theintell.com/opinion/20190828/lte...


In [ ]:
Enid_News=bbg_data[(bbg_data['Source']=="Enid News")].reset_index(drop=True)

In [ ]:
bad_url=[]
c=0
for url in Enid_News['URL']:
    print(c)
    c=c+1
    r=requests.get(url)
    if r.status_code!=200:
        bad_url.append(url)
        print(url)

In [ ]:
bad_url_frame=pd.DataFrame(columns=['URL'])
bad_url_frame['URL']=bad_url
bad_url_frame.to_csv('Enid_news_bad_url.csv')

In [3]:
bad_url=pd.read_csv('Enid_news_bad_url.csv')
bad_url.head(5)

,Unnamed: 0,URL
0,0,https://www.enidnews.com/oklahoma/news/arreste...
1,1,https://www.enidnews.com/oklahoma/news/budweis...
2,2,https://www.enidnews.com/sports/local_sports/b...
3,3,https://www.enidnews.com/sports/local_sports/s...
4,4,https://www.enidnews.com/sports/local_sports/o...


In [4]:
Enid_News=bbg_data[(bbg_data['Source']=="Enid News")&(bbg_data['URL'].isin(bad_url['URL'])==False)].reset_index(drop=True)
Enid_News.head()
print(len(Enid_News))

5580


In [ ]:
len(bad_url)

In [5]:
Enid_News['Category'].value_counts()

regular      5380
editorial     165
guest          35
Name: Category, dtype: int64

In [15]:
c=0
def get_metadata(url):
    global c
    print(c)
    c=c+1
    r = requests.get(url) 
    html_soup = BeautifulSoup(r.content, 'lxml')
    text = ''
    date= ''
    title=''
    author=''
    keywords=''
    description=''
    
    #To access article content
    if html_soup.findAll('div', attrs = {"itemprop" : "articleBody"}) is not None:
        temp_soup = html_soup.find('div', attrs = {"itemprop" : "articleBody"})
        try:
            soup = temp_soup.findAll('p')
            for s in soup:
                text = text + ' ' + s.get_text()
        except:
            print(url)
    else:
        print('Content')
    content=text.replace(u"\xa0","")# to remove latin space
    
    
    #To acess article date
    if html_soup.find('meta',attrs={"itemprop":"dateCreated"}) is not None:
        date=html_soup.find('meta',attrs={"itemprop":"dateCreated"})["content"]
        date=date.split("T")[0]
    else:
        print("Date")
    
    
    #To acess the title   
    if html_soup.find('meta',attrs={"property":"og:title"})is not None:
        title=html_soup.find('meta',attrs={"property":"og:title"})["content"]
        if "|" in title:
            title=title.split("|")[0]
    else:
        print("Title")
    title=title.replace(u"\xa0","")
    print(title)
    
    #To acess the author
    if html_soup.find('meta',attrs={"name":"author"}) is not None:
        author=html_soup.find('meta',attrs={"name":"author"})["content"]
        if "|" in author:
            author=author.split("|")[0]
    else:
        print("author")
    author=author.replace(u"\xa0","")
    
    
    #To access keywords
    if html_soup.find('meta',attrs={"name":"news_keywords"}) is not None:
        keywords=html_soup.find('meta',attrs={"name":"news_keywords"})["content"]
    else:
        print("Keywords")
    keywords=keywords.replace(u"\xa0","")
    
    
    #To access general description of the article
    if html_soup.find('meta',attrs={"name":"description"}) is not None:
        description=html_soup.find('meta',attrs={"name":"description"})["content"]
    else:
        print("Description")
    description=description.replace(u"\xa0","") 
    
    
    return (content,date,title,author,keywords,description)

In [16]:
Enid_News['Content'],Enid_News['Date'],Enid_News['Title'],Enid_News['Author'],Enid_News['Keywords'],Enid_News['Description'] = zip(*Enid_News['URL'].apply(get_metadata))

0
EPD: Many young drivers violate license rules
1
EDITORIAL: There's plenty to do in Enid this weekend
2
It's Gold Star Family Remembrance Week this week
3
Chisholm, Waukomis hopeful about regionals
4
Smithson 'a Trojan to the core'
5
Fairview's Ramay ranked No. 1 punter in the country
6
18th annual powwow set to honor Black Indian culture
7
EDITORIAL: There's plenty to do in Enid this weekend
8
Updated: Gatehouse Media buys The Oklahoman; layoffs expected
9
Armed robbery suspect brandished weapon before being shot, killed, police say
10
Updated: Westbound lane of U.S. 412 reopens after semi jack-knifes
11
Updated: Gatehouse Media buys The Oklahoman; layoffs expected
12
Two Woods County women charged with child neglect
13
Quakes reported near Coyle, Enid
14
Saturday street closures announced for Great Land Run race
15
Hennessey man charged with making bomb threat
16
Hennessey Wine & Chocolate festival Saturday
17
Armed robbery suspect brandished weapon before being shot, killed, police

Recent weather delays Cleveland work
155
Final meeting Thursday for Vance Join Land Use study
156
Mural to honor longtime EHS teacher
157
Free smoke alarms to be offered during "Sound the Alarm"
158
YMCA to host teen night Oct. 20; volunteers needed
159
Enid residents charged with child abuse felonies
160
Inhofe praises Collins decision, says Kavanaugh demonstrators 'not real people'
161
UPDATED: Enid loses to Edmond Santa Fe 53-7
162
UPDATED: Enid loses to Edmond Santa Fe 53-7
163
Dozens turn out for meeting to discuss Forgotten Ministries' transitional house proposal
164
Seat open on Waukomis Board of Education
165
Campbell: Nothing beats OU-Texas
166
Drummond falls in title game
167
Roundup: Washington leads Pioneer over Ringwood
168
Turnovers costly as Plainsmen fall 53-7
169
OBA remains undefeated with 33-13 defeat of Okeene
170
EPD: Man critical after being shot by homeowner
171
COLUMN: Our ever-changing history: Everything we do in live matters
172
Questions surround OU defense 

'Monarch Meetup' scheduled for Oct. 30
315
Zoe Kids Cafe to host free family night Oct. 26
316
Victim offering $500 reward in hit-and-run
317
Radio-TV for Monday 10-15
318
Prescott, Cowboys pack punch with pass in routing Jags
319
UT: Ehlinger has sprained shoulder
320
Iowa State continues to play giant killer
321
Campbell: Bad breaks can't keep Chisholm senior down
322
COLUMN: Our neighbor is far from boring
323
Enid man's murder conviction, sentence upheld in 2012 shooting
324
COLUMN: Our neighbor is far from boring
325
Power outage cancels classes and Enid High School
326
COLUMN: No fake news, just hard work and high standards
327
Campbell: Bad breaks can't keep Chisholm senior down
328
Time to panic: No quick fix for OSU Cowboys
329
Lucas endorses Stitt for governor
330
AG allows more time for wind farm plan
331
Updated: Enid ties record low Tuesday, breaks record Monday
332
AG calls on FCC for new rules involving scams and robocalls
333
Digital era is the test of Time
334
School b

Many opioid addicts shut out of sober-living homes
472
Thursday event to focus on Marsy's Law
473
A daring challenge
474
AFA bound Clark anxious for 3rd trip to state cross country
475
Medford presents challenge for Covington-Douglas
476
Veterans Day parade organizers seeking participants
477
COLUMN: Count each and every one of your blessings
478
COLUMN: Singing a different immigration tune
479
Survey says: Consumers to spend a total of billions on holidays this winter
480
MAPC to consider lot split for barbecue restaurant
481
GCRW membership meeting set for Oct. 29
482
Water to be shut off Thursday in some areas
483
YMCA offering military appreciation discount in November
484
Northwestern's annual Fallfest coming soon
485
EDITORIAL: We're now accepting Pillar of the Plains nominees
486
Crosswhite Hader, Scott face off House District 41 seat
487
Machine guns, shooting matches encroaching election
488
Ruthenberg: Barry Sanders gave Lions fans a 10-year thrill
489
Cowboys excited to cele

Pioneer hosts Chiefs for B-2 crown
618
Phillips reaches milestone, but EHS falls 76-0 in season finale
619
COLUMN: What did I just watch?
620
What's 2 Do in Enid, area
621
AAA alerting drivers to changes in commute following Daylight Saving Time
622
COLUMN: What did I just watch?
623
COLUMN: A taste sweeter than honey: Coming to Christ's banquet through grace
624
EHS Plainsmen head football coach steps down
625
EDITORIAL: The Era of Bad Feelings: Attack ads are destructive, contribute to voter apathy
626
Oklahomans to decide state races at the polls Nov. 6
627
Waukomis police seeking truck in Friday hit-and-run
628
Portion of Cambridge closing for repairs
629
Chestnut to close Wednesday for overpass work
630
Candidates, volunteers urged to keep signs from littering highways
631
YFS's Project Santa begins Monday
632
County resident doesn't miss chance to vote
633
Legislators act as 'super donors,' sending funds to other candidates
634
Campbell: Shamburg too busy to miss coaching
635
Are

...

Without federal aid, states help themselves
5040
Former Vance pilot remembers sister among Liberia Martyrs
5041
Invasive species a threat to farms
5042
COLUMN: One nation, under God, indivisible — period
5043
COLUMN: Time to buy cheap options on low volatility
5044
EDITORIAL: Kudos to those doing work for our community, schools
5045
Ruthenberg: Fighting the new football world order
5046
Police seeking public's help identifying potential thief
5047
Greater Enid PLAC meeting to feature Sen. Roger Thompson
5048
EFD using grass rigs for medical calls, lift assists
5049
High donor registration rates lead to low wait times for transplants in Oklahoma
5050
COLUMN: Things looking up for delayed telescope
5051
Oklahoma's revamped defense passes first test with Houston victory
5052
EPD advises Oklahomans carrying permitless firearms
5053
Journey House to host fundraising dinner, program update Sept. 17
5054
Questions abound about McAlester officer-involved shooting
5055
Enid library recognizing 

COLUMN: Restoring Oklahoma's Earned Income Tax Credit necessary
5174
OBA volleyball defeats Chisholm for second time this season
5175
Free seminar to examine love through theology and psychology
5176
COLUMN: Restoring Oklahoma's Earned Income Tax Credit necessary
5177
Free seminar to examine love through theology and psychology
5178
Tonkawa's Full Moon Concert to feature pieces from 'The Great American Songbook'
5179
Cherokee Strip Celebration comes to downtown Enid this weekend
5180
Construction underway for Ponca casino near Perry
5181
Nominations open for Junior Welfare League Legacy Award
5182
English students learn to communicate better by teaching
5183
OKC VA to hold POW/MIA remembrance ceremony
5184
National group scrutinizes Stitt's religious leanings
5185
Hunter: 'I'm not done' with opioid fight
5186
VIDEO EMBED: Oklahoma AG Mike Hunter on pivotal role of the Box family during opioid trial
5187
Iris society's annual sale is Saturday
5188
Guest Editorial: Mourning the losses of

COLUMN: Seeking life's right balance
5305
With kids’ futures at risk, schools seek new ways to lower chronic absenteeism
5306
Former Lahoma mayor charged with conspiracy
5307
Man injured following early morning home invasion
5308
UPDATED: Man killed in accident involving train
5309
Camping in the Park coming soon
5310
Mooreland man hurt in collision with cow
5311
Enid man charged with failing to care for brown lab
5312
Hennessey Wine & Chocolate Festival set for Saturday
5313
Operation Christmas Child Project Leader Workshop set Saturday
5314
State appeals judge’s order in 'Innocent Man' case
5315
Number of abortions in U.S. falls to lowest level since 1973
5316
Longdale man dies in accident
5317
Saudis couldn't stop oil attack, even with top U.S. defenses
5318
D.A.: Okla. officer justified in Molotov cocktail shooting
5319
Weapons, phones, drugs are found
5320
Pastor uses church's status to avoid property taxes, investigation shows
5321
County roundup: Garber battles illness before fa

Police arrest man burglarizing vehicles on East Maple
5439
EEOC sues Black Forest Decor for disability discrimination
5440
Cats, kittens available at Saturday adoption event
5441
Water to be shut off on portion of Oak
5442
Man charged in home burglary, shooting incident
5443
COLUMN: In the face of darkness and lies, we need the light of brave youth
5444
Man arrested in connection to home invasion on Jefferson
5445
Advocates push to reinstate child seat belt laws
5446
Westmoore blanks Plainsmen 54-0
5447
COLUMN: Stressing out isn't what God wants for us
5448
EDITORIAL: Enid residents should collaborate with neighbors, police to fend off home invasions
5449
Enid Symphony Auxiliary to host murder mystery dinner fundraiser
5450
Zingg bringing boogie woogie to Enid
5451
What's 2 Do in Enid, area
5452
Man charged in home burglary, shooting incident
5453
EEOC sues Black Forest Decor for disability discrimination
5454
Man arrested in connection to home invasion on Jefferson
5455
COLUMN: Stress

New jail administrator could be hired next week


In [35]:
Enid_news1=Enid_News.copy()

In [36]:
Enid_news1['ind']=Enid_news1['URL'].apply(lambda x: 'guest' if 'guest' in x else('opinion' if 'opinion' in x else 'regular'))

In [38]:
Enid_News['Category']=Enid_news1['ind']

In [39]:
Enid_News.head()

,Source,Timestamp,Category,URL,Content,Date,Title,Author,Keywords,Description
0,Enid News,29-SEP-19-04.27.53.653303,regular,https://www.enidnews.com/oklahoma/news/epd-man...,Enid Police Department has seen a recent incr...,2018-09-26,EPD: Many young drivers violate license rules,Cass Rains Staff Writer,"license, motor vehicle, transports, highway, d...",Enid Police Department has seen a recent incre...
1,Enid News,29-SEP-19-05.41.10.760725,opinion,https://www.enidnews.com/opinion/go-and-see/ar...,“There’s nothing to do in Enid.” That’s a pop...,2018-09-27,EDITORIAL: There's plenty to do in Enid this w...,Enid News & Eagle Editorial Board,"oktoberfest, food, gastronomy, enology, tom ni...",“There’s nothing to do in Enid.” That’s a popu...
2,Enid News,29-SEP-19-05.41.09.696911,opinion,https://www.enidnews.com/opinion/it-s-gold-sta...,"By Sen. James Lankford, R-Okla. Each year bef...",2018-09-27,It's Gold Star Family Remembrance Week this week,Sen.James Lankford,"week, politics, military, remembrance, photo, ...","By Sen. James Lankford, R-Okla."
3,Enid News,29-SEP-19-05.49.26.759437,regular,https://www.enidnews.com/sports/local_sports/c...,Chisholm coach Kevin Burns is hopeful about h...,2018-09-27,"Chisholm, Waukomis hopeful about regionals",Bruce Campbell Staff Writer,"tournament, sport, baseball, kevin burns, phil...",Chisholm coach Kevin Burns is hopeful about hi...
4,Enid News,29-SEP-19-05.54.22.577748,regular,https://www.enidnews.com/sports/local_sports/s...,"ENID, Okla. — Senior Traber Smithson is savor...",2018-09-27,Smithson 'a Trojan to the core',Bruce Campbell,"jay mendenhall, sport, american football, wint...",Senior Traber Smithson is savoring Oklahoma Bi...


In [40]:
Enid_News.to_csv('Bloomberg_Enid_News.csv')